In [24]:
import numpy as np
import pandas as pd

import requests ## python -m pip install requests 
import json

In [37]:
def signature_extractor(file):
    
    # preprocessing data before 
            #calculating up/down regulated genes
        
    stat = pd.read_table(file, sep=" ")
    stat = stat.apply(pd.to_numeric)
    
    up_genes = []
    down_genes = []
    names = stat.columns
    
    # calculating up/down regulated genes
    
    if names[0] == "logFC" and names[2] == "PValue":
        up_genes = stat[(stat["logFC"] >= np.log(2)) & (stat["PValue"] <= 1e-3)].index
        down_genes = stat[(stat["logFC"] <= np.log(0.5)) & (stat["PValue"] <= 1e-3)].index
        
    if names[1] == "log2FoldChange" and names[5] == "padj":
        up_genes = stat[(stat["log2FoldChange"] >= np.log2(2)) & (stat["padj"] <= 1e-3)].index
        down_genes = stat[(stat["log2FoldChange"] <= np.log2(0.5)) & (stat["padj"] <= 1e-3)].index   
        
    up_genes = [str(gene) for gene in up_genes]
    down_genes = [str(gene) for gene in down_genes] 
    
    return up_genes, down_genes

In [38]:
stat = pd.read_table("~/Downloads/DE_heart_fb_deseq2_edger.txt", sep=" ")

In [173]:
up_genes, down_genes = signature_extractor("~/Downloads/DE_heart_fb_deseq2_edger.txt")

In [174]:
print(len(up_genes), len(down_genes))

8580 11196


In [175]:
class adj_list:
    
    def __init__(self, up_genes, down_genes):
        self.up_genes = up_genes
        self.down_genes = down_genes
        self.interactions_up = []
        self.interactions_down = []
        self.adjac_list_up = []
        self.adjac_list_down = []
        
    def API_request(self, mode, species = 9606):
        string_api_url = "https://string-db.org/api"
        output_format = "tsv-no-header"
        method = "network"

        ##
        ## Construct URL
        ##

        request_url = "/".join([string_api_url, output_format, method])

        ##
        ## Set parameters
        ##

        my_genes = {
                'up_genes': self.up_genes,
                'down_genes': self.down_genes
                }

        params = {

            "identifiers" : "%0d".join(my_genes[mode]), # your protein
            "species" : 9606, # species NCBI identifier 
            "caller_identity" : "www.awesome_app.org" # your app name

        }

        ##
        ## Call STRING
        ##

        response = requests.post(request_url, data=params)
        
        interactions = []
        
        for line in response.text.strip().split("\n"):

            l = line.strip().split("\t")
            print(l)
            #p1, p2 = l[2], l[3]
            ## filter the interaction according to experimental score
            #experimental_score = float(l[10])
            #interactions.append((p1, p2, experimental_score))
        if mode == 'up_genes':    
            self.interactions_up = interactions
        else:
            self.interactions_down = interactions
        
        return interactions
    
    def creating_adj_list(self, mode, score = 0.0):
        adja_list = []
        genes = {
                'up_genes': self.up_genes,
                'down_genes': self.down_genes
                }
        if mode == "up_genes":
            interactions = self.interactions_up
        else:
            interactions = self.interactions_down
        
        for ind_i, line in enumerate(genes[mode]):
            adja_list.append([])
            for  ind_j, col in enumerate(genes[mode]):
                for ind_k, inter in enumerate(interactions):
                    if line == inter[0] and col == inter[1] and float(inter[2]) >= score:
                        if adja_list[ind_i] != [] and adja_list[ind_i][-1] != inter[1]:
                            adja_list[ind_i].append(inter[1])
                        elif adja_list[ind_i] == []:
                            adja_list[ind_i].append(inter[1])
        
        if mode == 'up_genes':    
            self.adjac_list_up = adja_list
        else:
            self.adjac_list_down = adja_list
            
        return adja_list

In [176]:
up = adj_list(up_genes, down_genes)
up.API_request(mode="up_genes")
up.creating_adj_list(mode="up_genes", score=0.1)

['Error', 'ErrorMessage']
['input too large', 'We do not support networks larger than 2000 nodes. Please go back and reduce the number of proteins that you provide as input.']


[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [177]:
down = adj_list(up_genes, down_genes)
down.API_request(mode="down_genes")
down.creating_adj_list(mode="down_genes", score=0.1)

['Error', 'ErrorMessage']
['input too large', 'We do not support networks larger than 2000 nodes. Please go back and reduce the number of proteins that you provide as input.']


KeyboardInterrupt: 